<a href="https://colab.research.google.com/github/shjoo0407/political_disposition/blob/main/eun/Emotions_in_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
cd /content/drive/MyDrive/2023/sw공모전

/content/drive/MyDrive/2023/sw공모전


In [ ]:
import pandas as pd
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### data laod

In [ ]:
ls

 data/                    topic_modeling3.html   문재인_국힘.html
'df(preprocessing).csv'   topic_modeling.html    문재인_민주당.html
'df(윤석렬).csv'          TopicModeling.ipynb    박근혜_국힘.html
 __pycache__/             utils.py               박근혜_민주당.html
 summarizatio.ipynb       노무현_국힘.html       이명박_국힘.html
 topic_modeling2.html     노무현_민주당.html     이명박_민주당.html


In [ ]:
df = pd.read_csv('df(윤석렬).csv')

In [ ]:
df['target'].value_counts()

국민의힘    3733
민주당     3659
Name: target, dtype: int64

In [ ]:
df.head()

,Unnamed: 0,content,target,content2,content3
0,0,‘빗물을 나르는 고속도로’라 불리는 대심도 빗물터널 조금 전에 둘러보고 왔다. 신월...,국민의힘,빗물 고속도로 심도 빗물 터널 신월 빗물 전용 배수 시설 집중호우 주변 지역 물바다...,"['빗물', '고속도로', '심도', '빗물', '터널', '신월', '빗물', '..."
1,1,"오늘 한 언론에 보도된 무소속 박완주 의원의 검찰 공소장 안에는, 강력 성범죄자의 ...",국민의힘,오늘 언론 보도 무소속 박완주 의원 검찰 공소장 성범죄 행태 모습 의원 자신 보좌관...,"['오늘', '언론', '보도', '무소속', '박완주', '의원', '검찰', '..."
2,2,전국적인 집중호우로 많은 국민들이 희생되고 막대한 재산 피해가 발생했다. 당장 전면...,국민의힘,전국 집중호우 국민 희생 재산 피해 발생 당장 전면 복구 작업 상황 오늘 내일 전국...,"['전국', '집중호우', '국민', '희생', '재산', '피해', '발생', '..."
3,3,더불어민주당이 윤석열 정부의 ‘시행령 추진’에 본격적으로 제동을 걸고 있다. KBS...,국민의힘,더불어민주당 윤석열 정부 시행령 추진 제동 수신료 분리 징수 내용 방송법 시행령 검...,"['더불어민주당', '윤석열', '정부', '시행령', '추진', '제동', '수신..."
4,4,"그칠 줄 모르던 비가 잠시 소강상태에 접어들었지만, 폭우가 할퀴고 간 자리에 남겨진...",국민의힘,소강상태 폭우 자리 주민 아픔 주말 예고 장마 소식 피해 복구 폭염 자연재해 주민 ...,"['소강상태', '폭우', '자리', '주민', '아픔', '주말', '예고', '..."


In [ ]:
keywords =['이상민','탄핵','해임','책임자','안부','안전부',' 시행령',' 구청장',' 용산',' 건의안']

# filtered_df = df[df['content3'].apply(lambda text_list: any(keyword in text_list for keyword in keywords))]
filtered_df = df[df['content3'].apply(lambda text_list: sum(keyword in text_list for keyword in keywords) >= 2)]

In [ ]:
filtered_df['target'].value_counts()

민주당     172
국민의힘    148
Name: target, dtype: int64

### Emotions in segmentation (60)

In [ ]:
# 감정 라벨
label_id2label = {
    "0": "분노",
    "1": "툴툴대는",
    "2": "좌절한",
    "3": "짜증내는",
    "4": "방어적인",
    "5": "악의적인",
    "6": "안달하는",
    "7": "구역질 나는",
    "8": "노여워하는",
    "9": "성가신",
    "10": "슬픔",
    "11": "실망한",
    "12": "비통한",
    "13": "후회되는",
    "14": "우울한",
    "15": "마비된",
    "16": "염세적인",
    "17": "눈물이 나는",
    "18": "낙담한",
    "19": "환멸을 느끼는",
    "20": "불안",
    "21": "두려운",
    "22": "스트레스 받는",
    "23": "취약한",
    "24": "혼란스러운",
    "25": "당혹스러운",
    "26": "회의적인",
    "27": "걱정스러운",
    "28": "조심스러운",
    "29": "초조한",
    "30": "상처",
    "31": "질투하는",
    "32": "배신당한",
    "33": "고립된",
    "34": "충격 받은",
    "35": "가난한 불우한",
    "36": "희생된",
    "37": "억울한",
    "38": "괴로워하는",
    "39": "버려진",
    "40": "당황",
    "41": "고립된(당황한)",
    "42": "남의 시선을 의식하는",
    "43": "외로운",
    "44": "열등감",
    "45": "죄책감의",
    "46": "부끄러운",
    "47": "혐오스러운",
    "48": "한심한",
    "49": "혼란스러운(당황한)",
    "50": "기쁨",
    "51": "감사하는",
    "52": "신뢰하는",
    "53": "편안한",
    "54": "만족스러운",
    "55": "흥분",
    "56": "느긋",
    "57": "안도",
    "58": "신이 난",
    "59": "자신하는"
  }

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 모델과 토크나이저 로드
model_name = "hun3359/mdistilbertV3.1-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
texts = filtered_df['content']
texts[:3]

15     재난안전대책을 맡아서 각 부처별로 열심히 활동하고 계시면서 사고 예방과 주민 대피를...
17     중앙재난안전대책본부가 발표한 오늘 아침 6시 기준 호우 관련 인명피해 현황을 살펴보...
114    오늘 IAEA 사무총장이 방한해 주말 간 외교부 장관 및 원자력위원장과 면담을 가질...
Name: content, dtype: object

In [ ]:
from tqdm import tqdm

In [ ]:
# emotion_list = []

# for text in tqdm(texts):
#   # 토크나이징 및 모델 입력 형식으로 변환
#   inputs = tokenizer(text, return_tensors="pt", max_length=512)

#   # 모델 추론
#   with torch.no_grad():
#       outputs = model(**inputs)
#       logits = outputs.logits

#   # 결과 확인
#   probs = logits.softmax(dim=-1).tolist()[0]

#   emotion_probabilities = [(label_id2label[str(label_id)], prob) for label_id, prob in enumerate(probs)]
#   sorted_emotion_probabilities = sorted(emotion_probabilities, key=lambda x: x[1], reverse=True)

#   # print(text)

#   emotions = []
#   for emotion, prob in sorted_emotion_probabilities[:5]:
#     emotions.append(emotion)
#     # print(f"Emotion: {emotion}, Probability: {prob:.4f}")

#   emotion_list.append(emotions)

In [ ]:
from transformers import pipeline

sentiment_analyzer = pipeline("sentiment-analysis", model="hun3359/mdistilbertV3.1-sentiment")

emotion_list = []

for text in tqdm(texts):
  result = sentiment_analyzer(text, max_length=512, truncation=True)
  emotion_list.append(result[0]['label'])

  # 결과 출력
  # print("Text:", text)
  # print("Sentiment:", result[0]['label'])
  # print(text)

100%|██████████| 320/320 [02:43<00:00,  1.95it/s]


In [ ]:
filtered_df['emotions2'] = emotion_list

<ipython-input-74-e6548c30196f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['emotions2'] = emotion_list


In [ ]:
filtered_df['emotions2'].head(5)

15       고립된(당황한)
17       고립된(당황한)
114    혼란스러운(당황한)
153    혼란스러운(당황한)
161      고립된(당황한)
Name: emotions2, dtype: object

In [ ]:
from collections import Counter

민주당_wordcnt = Counter(filtered_df[filtered_df['target']=='민주당']['emotions2'])

normalized_emotion_frequencies = {emotion: count / len(filtered_df[filtered_df['target']=='민주당']['emotions2'])*100 for emotion, count in 민주당_wordcnt.items()}

민_sorted_emotions = sorted(normalized_emotion_frequencies, key=normalized_emotion_frequencies.get, reverse=True)
print(민_sorted_emotions)

['고립된(당황한)', '혼란스러운(당황한)', '안달하는', '당황', '감사하는', '혐오스러운', '희생된', '억울한', '눈물이 나는']


In [ ]:
from collections import Counter

국민의힘_wordcnt = Counter(filtered_df[filtered_df['target']=='국민의힘']['emotions2'])

normalized_emotion_frequencies = {emotion: count / len(filtered_df[filtered_df['target']=='국민의힘']['emotions2'])*100 for emotion, count in 국민의힘_wordcnt.items()}
selected_emotions = {emotion: percentage for emotion, percentage in normalized_emotion_frequencies.items()if percentage >= 10}

국_sorted_emotions = sorted(normalized_emotion_frequencies, key=normalized_emotion_frequencies.get, reverse=True)
print(국_sorted_emotions)

['고립된(당황한)', '혼란스러운(당황한)', '안달하는', '당황', '혐오스러운', '구역질 나는', '희생된']


In [ ]:
set(민_sorted_emotions)- set(국_sorted_emotions)

set()

In [ ]:
set(국_sorted_emotions)- set(민_sorted_emotions )

{'성가신', '신이 난', '자신하는'}

### Emotions in segmentation(3)

In [ ]:
id2label: {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  }

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 모델과 토크나이저 로드
model_name = "snunlp/KR-FinBert-SC"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
texts = filtered_df['content']

emotion_list = []
for text in tqdm(texts):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

    # 모델 추론
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 확률 계산 및 정규화
    emotion_probs = torch.softmax(logits, dim=-1)[0]

    # 가장 높은 확률을 가진 클래스 선택
    max_prob, max_prob_index = torch.max(emotion_probs, dim=0)
    max_prob_emotion = model.config.id2label[max_prob_index.item()]

    emotion_list.append(max_prob_emotion)

    # print(f"Emotion: {max_prob_emotion}, Probability: {max_prob.item():.2f}%")

100%|██████████| 1376/1376 [35:42<00:00,  1.56s/it]


In [ ]:
filtered_df['emotions2'] = emotion_list

<ipython-input-272-e6548c30196f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['emotions2'] = emotion_list


In [ ]:
from collections import Counter

민주당_wordcnt = Counter(filtered_df[filtered_df['target']=='민주당']['emotions2'])

normalized_emotion_frequencies = {emotion: count / len(filtered_df[filtered_df['target']=='민주당']['emotions2'])*100 for emotion, count in 민주당_wordcnt.items()}

민_sorted_emotions = sorted(normalized_emotion_frequencies, key=normalized_emotion_frequencies.get, reverse=True)
print(민_sorted_emotions)

['neutral', 'negative']


In [ ]:
from collections import Counter

국민의힘_wordcnt = Counter(filtered_df[filtered_df['target']=='국민의힘']['emotions2'])

normalized_emotion_frequencies = {emotion: count / len(filtered_df[filtered_df['target']=='국민의힘']['emotions2'])*100 for emotion, count in 국민의힘_wordcnt.items()}
selected_emotions = {emotion: percentage for emotion, percentage in normalized_emotion_frequencies.items()if percentage >= 10}

국_sorted_emotions = sorted(normalized_emotion_frequencies, key=normalized_emotion_frequencies.get, reverse=True)
print(국_sorted_emotions)

['neutral', 'negative']
